In [5]:
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import json
from datetime import datetime

# Load the trained model
MODEL_PATH = "../model/poker_card_detection/weights/best.pt"
model = YOLO(MODEL_PATH)

# Create result directories
RESULT_DIR = "../result/poker-card-detection-result"
IMAGE_RESULT_DIR = os.path.join(RESULT_DIR, "image")
VIDEO_RESULT_DIR = os.path.join(RESULT_DIR, "video")
os.makedirs(IMAGE_RESULT_DIR, exist_ok=True)
os.makedirs(VIDEO_RESULT_DIR, exist_ok=True)

def process_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    results = model(img)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(img)
    
    detections = []
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            label = f"{result.names[int(cls)]} {conf:.2f}"
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            ax.text(x1, y1, label, color='white', fontweight='bold', bbox=dict(facecolor='red', alpha=0.5))
            
            detections.append({
                "class": result.names[int(cls)],
                "confidence": float(conf),
                "bbox": [float(x) for x in [x1, y1, x2, y2]]
            })
    
    plt.axis('off')
    
    plot_filename = os.path.join(IMAGE_RESULT_DIR, f"detection_{os.path.basename(image_path)}")
    plt.savefig(plot_filename)
    plt.close()
    
    return detections, plot_filename

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    output_path = os.path.join(VIDEO_RESULT_DIR, f"detection_{os.path.basename(video_path)}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame)
        
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            
            for box, cls, conf in zip(boxes, classes, confidences):
                x1, y1, x2, y2 = map(int, box)
                label = f"{result.names[int(cls)]} {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        out.write(frame)
        
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    out.release()
    
    print(f"Video processing completed. Output saved to {output_path}")
    return output_path

def main():
    # Process images
    test_image_dir = "../data/raw/poker-cards-v2-voc/test"
    test_images = [os.path.join(test_image_dir, img) for img in os.listdir(test_image_dir) if img.endswith('.jpg')][:5]
    
    image_results = []
    for img_path in test_images:
        print(f"Processing image: {os.path.basename(img_path)}")
        detections, plot_path = process_image(img_path)
        image_results.append({
            "image_path": img_path,
            "detections": detections,
            "plot_path": plot_path
        })
    
    # Process video
    video_path = "../video/poker-card-detection-video/1.mp4"
    print(f"Processing video: {os.path.basename(video_path)}")
    output_path = process_video(video_path)
    video_results = [{
        "video_path": video_path,
        "output_path": output_path
    }]
    
    # Save results to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    json_filename = os.path.join(RESULT_DIR, f"detection_results_{timestamp}.json")
    with open(json_filename, 'w') as f:
        json.dump({
            "image_results": image_results,
            "video_results": video_results
        }, f, indent=2)
    
    print(f"Model testing completed. Results saved to {json_filename}")

if __name__ == "__main__":
    main()

Processing image:  (1).jpg

0: 128x160 1 3 Hearts, 122.3ms
Speed: 5.1ms preprocess, 122.3ms inference, 5.0ms postprocess per image at shape (1, 3, 128, 160)
Processing image:  (10).jpg

0: 128x160 1 2 Hearts, 1 3 Hearts, 1 4 Hearts, 1 A Spades, 91.6ms
Speed: 1.0ms preprocess, 91.6ms inference, 3.0ms postprocess per image at shape (1, 3, 128, 160)
Processing image:  (100).jpg

0: 128x160 1 2 Hearts, 1 4 Spades, 116.4ms
Speed: 4.2ms preprocess, 116.4ms inference, 2.0ms postprocess per image at shape (1, 3, 128, 160)
Processing image:  (101).jpg

0: 128x160 1 3 Hearts, 2 4 Heartss, 1 A Trefoils, 97.3ms
Speed: 1.0ms preprocess, 97.3ms inference, 4.0ms postprocess per image at shape (1, 3, 128, 160)
Processing image:  (102).jpg

0: 128x160 1 3 Hearts, 1 4 Hearts, 1 A Trefoils, 87.1ms
Speed: 2.0ms preprocess, 87.1ms inference, 2.0ms postprocess per image at shape (1, 3, 128, 160)
Processing video: 1.mp4

0: 160x96 1 A Hearts, 1 A Trefoils, 99.9ms
Speed: 2.0ms preprocess, 99.9ms inference, 3.